<h1>Importing Libraries</h1>

In [ ]:
import os
import gc
import re

import cv2
import math
import random
import numpy as np
import scipy as sp
import pandas as pd

import tensorflow as tf
from IPython.display import SVG
from keras.utils import plot_model
import tensorflow.keras.layers as L
from keras.utils import model_to_dot
import tensorflow.keras.backend as K
from tensorflow.keras.models import Model
from kaggle_datasets import KaggleDatasets
from tensorflow.keras.applications import DenseNet201, InceptionV3, ResNet50V2, InceptionResNetV2, VGG19, VGG16

import seaborn as sns
from tqdm import tqdm
import matplotlib.cm as cm
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

tqdm.pandas()
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

np.random.seed(0)
tf.random.set_seed(0)

import warnings
warnings.filterwarnings("ignore")


import cv2 as cv
from skimage import filters
from skimage import morphology

import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
import matplotlib.pyplot as plt

from kaggle_datasets import KaggleDatasets
import tensorflow as tf
print(f"Tensorflow version: {tf.__version__}")
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score

SEED = 5

<h1>Loading data</h1>

In [ ]:
EPOCHS = 40
SAMPLE_LEN = 1821
INPUT_PATH = '/kaggle/input/plant-pathology-2020-fgvc7/'
IMG_PATH = INPUT_PATH + 'images/'
TRAIN_DATA = INPUT_PATH + 'train.csv'
TEST_DATA = INPUT_PATH + 'test.csv'
SAMPLE_SUB = INPUT_PATH + 'sample_submission.csv'


In [ ]:
train_df = pd.read_csv(TRAIN_DATA)
test_df = pd.read_csv(TEST_DATA)
sampleSubmission_df = pd.read_csv(SAMPLE_SUB)

<h1>EDA</h1>

In [ ]:
EDA_IMG_SHAPE = (512,256)

def getImage(image_id,SHAPE=EDA_IMG_SHAPE):
    img = cv.imread(IMG_PATH + image_id + '.jpg')
    img = cv.resize(img,SHAPE)
    img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
    
    return img

In [ ]:
%%time

healthy = [getImage(image_id) for image_id in train_df[train_df['healthy']==1].iloc[:,0]]

multiple_diseases = [getImage(image_id) for image_id in train_df[train_df['multiple_diseases']==1].iloc[:,0]]

rust = [getImage(image_id) for image_id in train_df[train_df['rust']==1].iloc[:,0]]

scab = [getImage(image_id) for image_id in train_df[train_df['scab']==1].iloc[:,0]]

In [ ]:
classes = {'healthy':healthy, 'multiple_diseases':multiple_diseases, 'rust':rust, 'scab': scab} 

In [ ]:
#Exploratory Data Analysis

In [ ]:
def plotlyDataFrame(df,title):
    
    fig = go.Figure(data=[go.Table(
    header = dict(values = df.columns),
    cells = dict(values = [df[col] for col in df.columns]))])
    
    fig.update_layout(
        title = title)
    
    fig.show()

In [ ]:
#Train DataFrame

In [ ]:
plotlyDataFrame(train_df.iloc[:15,:],'Train Data')

In [ ]:
#Test DataFrame

In [ ]:
plotlyDataFrame(test_df.iloc[:15,:],'Test Data')

In [ ]:
#Sample Submission File
plotlyDataFrame(sampleSubmission_df.iloc[:15,:], 'Sample Submission')

<h2>Class distributions</h2>

In [ ]:


fig = go.Figure(data=[go.Pie(labels=train_df.columns[1:],
                             values=[np.sum(train_df[col]) for col in train_df.columns[1:]])])

fig.update_traces(hoverinfo='label+percent',
                  textinfo='value',
                  textfont_size=20,
                  marker=dict(line=dict(color='#000000', width=2)))

fig.update_layout(title_text="Target Distribution of Training-Data ")

fig.show()

In [ ]:
## Image Augmentation
#Image data augmentation is a technique that can be used to artificially expand the size of a training dataset by creating modified versions of images in the dataset.

#Training deep learning neural network models on more data can result in more skillful models, and the augmentation techniques can create variations of the images that can improve the ability of the fit models to generalize what they have learned to new images.

#Transforms include a range of operations from the field of image manipulation, such as shifts, flips, zooms, and much more.

In [ ]:
#Sample Image

def getRandomImage():
    return random.choice(classes[random.choice(train_df.columns[2:])])

In [ ]:
img = getRandomImage()

In [ ]:
fig = go.Figure(go.Image(z=img))

fig.update_layout(title_text="Smaple Image")

fig.show()

In [ ]:
#Image Augmentation With ImageDataGenerator
#We can use this to generate augmented images using different transformations like:

#Vertical Flipping
#Horizontal Flipping
#Shifted Images
#Rotated Images
#Zoomed Images
#Images with different Brightness levels etc.
#Plus point of using this is that it will generate images in runtime (While training a model). It means we are not supposed to store this augmented images.

In [ ]:
img = np.expand_dims(img,axis=0)

In [ ]:
%%time

generator = tf.keras.preprocessing.image.ImageDataGenerator(vertical_flip=True,
                                                            horizontal_flip=True,
                                                            brightness_range=[0.5,1.5],
                                                            zoom_range=[0.5,1.1])

iterator = generator.flow(img,batch_size=1)



fig, ax = plt.subplots(nrows=5, ncols=3, figsize=(15,10))

ax[0,0].imshow(img[0])
ax[0,0].set_title("Sample Image",fontsize=10)
ax[0,0].set_xticks([])
ax[0,0].set_yticks([])


for i in range(1,15):
    
    image = iterator.next()[0].astype('uint8')
    
    ax[i//3,i%3].imshow(image)
    ax[i//3,i%3].set_xticks([])
    ax[i//3,i%3].set_yticks([])

fig.suptitle("Augmented Images of the sample image",fontsize=20)

plt.show()

In [ ]:
sampleImg = getRandomImage()

In [ ]:
def convertToHSV(img):
    return cv.cvtColor(img,cv.COLOR_RGB2HSV_FULL)

In [ ]:
#ROI (Region Of Interest) Selection


#To select the ROI, I have used Canny edge detection to detect the edges of the leaves 
#and to find the edges of the rectangle ROI, I have used this: getROI method.


In [ ]:

def getROI(img):
    # convert the image to the gray-scale image
    gray = cv.cvtColor(img,cv.COLOR_RGB2GRAY)
    
    # Detect the edges in the image using canny edge detection
    edged = cv.Canny(gray,150,200)
    
    xm = img.shape[1]//2    # Middle coordinate of the x-axis (width of the image)
    ym = img.shape[0]//2    # Middle coordinate of the y-axis (height of the image)
    
    # to find the bottom-y coordinate to the Rectangle-ROI
    for i in range(img.shape[0]-1,-1,-1):
        if np.sum(edged[i,xm-5:xm+5])!=0:
            y_bottom = np.where(i+10<img.shape[0]-1,i+10,img.shape[0]-2)
            break
            
    # to find the top-y coordinate to the Rectangle-ROI
    for i in range(img.shape[0]):
        if np.sum(edged[i,xm-5:xm+5])!=0:
            y_top = np.where(i-10>1,i-10,2)
            break
    
    # to find the top-x coordinate to the Rectangle-ROI
    for i in range(img.shape[1]):
        if np.sum(edged[ym-5:ym+5,i])!=0:
            x_top = np.where(i-10>1,i-10,2)
            break
            
    # to find the bottom-x coordinate to the Rectangle-ROI
    for i in range(img.shape[1]-1,-1,-1):
        if np.sum(edged[ym-5:ym+5,i])!=0:
            x_bottom = np.where(i+10<img.shape[1]-1,i+10,img.shape[1]-2)
            break

    return edged,(x_top,y_top,x_bottom,y_bottom)

In [ ]:
fig, ax = plt.subplots(nrows=3, ncols=3, figsize=(20, 13))

for i in range(3):
    orignal = getRandomImage()
    edged, coordinates = getROI(orignal)
    
    roi = orignal.copy()
    
    (x_top,y_top,x_bottom,y_bottom) = coordinates
    
    roi[y_top-2:y_top,x_top:x_bottom+1] = [255,0,0]        # Top-edge
    roi[y_bottom:y_bottom+2,x_top:x_bottom+1] = [255,0,0]  # Bottom-edge
    roi[y_top:y_bottom+1,x_top-2:x_top] = [255,0,0]        # Left-edge
    roi[y_top:y_bottom+1,x_bottom:x_bottom+2] = [255,0,0]  # Right-edge
    
    ax[i,0].imshow(orignal)
    ax[i,0].set_title('Original Image', fontsize=15)
    ax[i,1].imshow(edged, cmap='gray')
    ax[i,1].set_title('Detected Edges', fontsize=15)
    ax[i,2].imshow(roi)
    ax[i,2].set_title('ROI', fontsize=15)
    
fig.suptitle("ROI selection using Canny Edge Detection",fontsize=20)
    
plt.show()

In [ ]:
#ROI selection using Watershed Transformation



In [ ]:
%%time

fig, ax = plt.subplots(nrows=6, ncols=3, figsize=(15,15))

for i in range(3):
    orignal = getRandomImage()
#     blur = cv.bilateralFilter(orignal,9,75,75)
    
    gray = cv.cvtColor(orignal,cv.COLOR_RGB2GRAY)
    sobel = filters.sobel(gray)

    blurred = filters.gaussian(sobel, sigma=2.0)
    
    ym = blurred.shape[0]//2
    xm = blurred.shape[1]//2
    
    markers = np.zeros(blurred.shape,dtype=np.int)
    # using corners of the image as background
    markers[0,0:2*xm] = 1
    markers[2*ym-1,0:2*xm] = 1
    markers[0:2*ym,0] = 1
    markers[0:2*ym,2*xm-1] = 1
    
    # using middle part of the image as foreground
    markers[ym-50:ym+50,xm-20:xm+20] = 2
    
    mask = morphology.watershed(blurred, markers)
    
    ax[0,i].imshow(orignal)
    ax[0,i].set_title('Original Image', fontsize=12)
    
    ax[1,i].imshow(gray, cmap='gray')
    ax[1,i].set_title('Gray Image', fontsize=12)
    
    ax[2,i].imshow(sobel, cmap='gray')
    ax[2,i].set_title('After Sobel Filter', fontsize=12)
    
    ax[3,i].imshow(blurred, cmap='gray')
    ax[3,i].set_title('Blurred Image', fontsize=12)
    
    ax[4,i].imshow(mask, cmap='gray')
    ax[4,i].set_title('Mask', fontsize=12)
    
    orignal[mask==1,:] = [0,0,0]
    
    ax[5,i].imshow(orignal)
    ax[5,i].set_title('Segmented Image', fontsize=12)
    

for i in range(6):
    for j in range(3):
        ax[i,j].set_xticks([])
        ax[i,j].set_yticks([])
    
fig.suptitle("Image Segmentation (ROI selection) using Watershed Transformation",fontsize=20)
    
plt.show()

In [ ]:
#HSV Conversion

fig, ax = plt.subplots(nrows=3, ncols=2, figsize=(10, 10))

for i in range(3):
    orignal = getRandomImage()
    hsv = convertToHSV(orignal)
    
    ax[i,0].imshow(orignal)
    ax[i,0].set_title('Original Image', fontsize=15)
    ax[i,1].imshow(hsv, cmap='gray')
    ax[i,1].set_title('HSV Image', fontsize=15)
    
fig.suptitle("RGB to HSV Conversion",fontsize=20)
    
plt.show()

In [ ]:
##Gray Scale Conversion
fig, ax = plt.subplots(nrows=3, ncols=2, figsize=(10, 10))

for i in range(3):
    orignal = getRandomImage()
    gray = cv.cvtColor(orignal,cv.COLOR_RGB2GRAY)
    
    ax[i,0].imshow(orignal)
    ax[i,0].set_title('Original Image', fontsize=15)
    ax[i,1].imshow(gray, cmap='gray')
    ax[i,1].set_title('Gray Image', fontsize=15)
    
fig.suptitle("RGB to Gray Scale Conversion",fontsize=20)

plt.show()

<h2>Image Examples</h2>

In [ ]:
#Function for showing image
def show_images(image_ids):
    
    col = 5
    row = min(len(image_ids) // col, 5)
    
    fig, ax = plt.subplots(row, col, figsize=(16, 8))
    ax = ax.flatten()

    for i, image_id in enumerate(image_ids):
        image = cv2.imread(IMAGE_PATH + '/{}.jpg'.format(image_id))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        ax[i].set_axis_off()
        ax[i].imshow(image)
        ax[i].set_title(image_id)

<h3>Random samples</h3>

<h1>Setup TPU Config</h1>

In [ ]:
AUTO = tf.data.experimental.AUTOTUNE
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
strategy = tf.distribute.experimental.TPUStrategy(tpu)

BATCH_SIZE = 16 * strategy.num_replicas_in_sync
GCS_DS_PATH = KaggleDatasets().get_gcs_path()

<h1>Load labels and paths</h1>

In [ ]:
sub = pd.read_csv(SAMPLE_SUB)
test_data = pd.read_csv(TEST_DATA)
train_data = pd.read_csv(TRAIN_DATA)

In [ ]:
def format_path(st):
    return GCS_DS_PATH + '/images/' + st + '.jpg'

test_paths = test_data.image_id.apply(format_path).values
train_paths = train_data.image_id.apply(format_path).values

train_labels = np.float32(train_data.loc[:, 'healthy':'scab'].values)
train_paths, valid_paths, train_labels, valid_labels =\
train_test_split(train_paths, train_labels, test_size=0.15, random_state=42)

In [ ]:
def decode_image(filename, label=None, image_size=(512, 512)):
    bits = tf.io.read_file(filename)
    image = tf.image.decode_jpeg(bits, channels=3)
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.image.resize(image, image_size)
    
    if label is None:
        return image
    else:
        return image, label

def data_augment(image, label=None): # Data augmentations
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_saturation(image, 0.7, 1.3)
    image = tf.image.random_contrast(image, 0.8, 1.2)
    imgae = tf.image.random_brightness(image, 0.1)
    
    if label is None:
        return image
    else:
        return image, label

<h1>Creating Dataset objects</h1>

In [ ]:
train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((train_paths, train_labels))
    .map(decode_image, num_parallel_calls=AUTO)
    .map(data_augment, num_parallel_calls=AUTO)
    .repeat()
    .shuffle(512)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)

valid_dataset = (
    tf.data.Dataset
    .from_tensor_slices((valid_paths, valid_labels))
    .map(decode_image, num_parallel_calls=AUTO)
    .batch(BATCH_SIZE)
    .cache()
    .prefetch(AUTO)
)

test_dataset = (
    tf.data.Dataset
    .from_tensor_slices(test_paths)
    .map(decode_image, num_parallel_calls=AUTO)
    .batch(BATCH_SIZE)
)

<h1>Defining Learning-Rate Scheduler</h1>

In [ ]:
def build_lrfn(lr_start=0.00001, lr_max=0.00005, 
               lr_min=0.00001, lr_rampup_epochs=5, 
               lr_sustain_epochs=0, lr_exp_decay=.8):
    lr_max = lr_max * strategy.num_replicas_in_sync

    def lrfn(epoch):
        if epoch < lr_rampup_epochs:
            lr = (lr_max - lr_start) / lr_rampup_epochs * epoch + lr_start
        elif epoch < lr_rampup_epochs + lr_sustain_epochs:
            lr = lr_max
        else:
            lr = (lr_max - lr_min) *\
                 lr_exp_decay**(epoch - lr_rampup_epochs\
                                - lr_sustain_epochs) + lr_min
        return lr
    return lrfn

<h1>Defining hyperparameters of fit</h1>

In [ ]:
lrfn = build_lrfn()
STEPS_PER_EPOCH = train_labels.shape[0] // BATCH_SIZE
lr_schedule = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose=1)

<h1>Function for visualizing training and validation accuracy</h1>

In [ ]:
def display_training_curves(training, validation, yaxis):
    if yaxis == "loss":
        ylabel = "Loss"
        title = "Loss vs. Epochs"
    else:
        ylabel = "Accuracy"
        title = "Accuracy vs. Epochs"
        
    fig = go.Figure()
        
    fig.add_trace(
        go.Scatter(x=np.arange(1, EPOCHS+1), mode='lines+markers', y=training, marker=dict(color="dodgerblue"),
               name="Train"))
    
    fig.add_trace(
        go.Scatter(x=np.arange(1, EPOCHS+1), mode='lines+markers', y=validation, marker=dict(color="darkorange"),
               name="Val"))
    
    fig.update_layout(title_text=title, yaxis_title=ylabel, xaxis_title="Epochs", template="plotly_white")
    fig.show()

<h1>Modelling</h1>

<h2>1. DenseNet</h2>

In [ ]:
#Setting the model to train in TPU
with strategy.scope():
    model = tf.keras.Sequential([DenseNet201(input_shape=(512, 512, 3),
                                             weights='imagenet',
                                             include_top=False),
                                 L.GlobalAveragePooling2D(),
                                 L.Dense(train_labels.shape[1],
                                         activation='softmax')])
        
    model.compile(optimizer='adam',
                  loss = 'categorical_crossentropy',
                  metrics=['categorical_accuracy'])
    model.summary()

In [ ]:
#Fundamental Block
SVG(tf.keras.utils.model_to_dot(Model(model.layers[0].input, model.layers[0].layers[13].output), dpi=70).create(prog='dot', format='svg'))

In [ ]:
#Model Architecture
SVG(tf.keras.utils.model_to_dot(model, dpi=70).create(prog='dot', format='svg'))

In [ ]:
#Training
history = model.fit(train_dataset,
                    epochs=EPOCHS,
                    callbacks=[lr_schedule],
                    steps_per_epoch=STEPS_PER_EPOCH,
                    validation_data=valid_dataset)

In [ ]:
print("Loss of the model is - " , model.evaluate(valid_dataset)[1]*100, "%")
print("Accuracy of the model is - " , model.evaluate(train_dataset)[1]*100 , "%")

In [ ]:
model.save('DenseNet.hdf5')

In [ ]:
#Visualizing train and valid accuracy
display_training_curves(
    history.history['categorical_accuracy'], 
    history.history['val_categorical_accuracy'], 
    'accuracy')

In [ ]:
#Visualizing train and valid loss
display_training_curves(
    history.history['loss'], 
    history.history['val_loss'], 
    'loss')

In [ ]:
#plotting training values
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

acc = history.history['categorical_accuracy']
val_acc = history.history['val_categorical_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)

#accuracy plot
plt.plot(epochs, acc, color='blue', label='Training Accuracy')
plt.plot(epochs, val_acc, color='red', label='Validation Accuracy')
plt.title('Training and Validation Accuracy with DenseNet')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend()

plt.figure()
#loss plot
plt.plot(epochs, loss, color='blue', label='Training Loss')
plt.plot(epochs, val_loss, color='red', label='Validation Loss')
plt.title('Training and Validation Loss with DenseNet')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

In [ ]:
#Prediction
probs_dnn = model.predict(test_dataset, verbose=1)
sub.loc[:, 'healthy':] = probs_dnn
sub.to_csv('submission_densenet.csv', index=False)
sub.head()

#LB:0.96792

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt # plotting
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [ ]:
# Distribution graphs (histogram/bar graph) of column data
def plotPerColumnDistribution(df, nGraphShown, nGraphPerRow):
    nunique = df.nunique()
    df = df[[col for col in df if nunique[col] > 1 and nunique[col] < 50]] # For displaying purposes, pick columns that have between 1 and 50 unique values
    nRow, nCol = df.shape
    columnNames = list(df)
    nGraphRow = (nCol + nGraphPerRow - 1) / nGraphPerRow
    plt.figure(num = None, figsize = (6 * nGraphPerRow, 8 * nGraphRow), dpi = 80, facecolor = 'w', edgecolor = 'k')
    for i in range(min(nCol, nGraphShown)):
        plt.subplot(nGraphRow, nGraphPerRow, i + 1)
        columnDf = df.iloc[:, i]
        if (not np.issubdtype(type(columnDf.iloc[0]), np.number)):
            valueCounts = columnDf.value_counts()
            valueCounts.plot.bar()
        else:
            columnDf.hist()
        plt.ylabel('counts')
        plt.xticks(rotation = 90)
        plt.title(f'{columnNames[i]} (column {i})')
    plt.tight_layout(pad = 1.0, w_pad = 1.0, h_pad = 1.0)
    plt.show()

In [ ]:
# Correlation matrix
def plotCorrelationMatrix(df, graphWidth):
    filename = df.dataframeName
    df = df.dropna('columns') # drop columns with NaN
    df = df[[col for col in df if df[col].nunique() > 1]] # keep columns where there are more than 1 unique values
    if df.shape[1] < 2:
        print(f'No correlation plots shown: The number of non-NaN or constant columns ({df.shape[1]}) is less than 2')
        return
    corr = df.corr()
    plt.figure(num=None, figsize=(graphWidth, graphWidth), dpi=80, facecolor='w', edgecolor='k')
    corrMat = plt.matshow(corr, fignum = 1)
    plt.xticks(range(len(corr.columns)), corr.columns, rotation=90)
    plt.yticks(range(len(corr.columns)), corr.columns)
    plt.gca().xaxis.tick_bottom()
    plt.colorbar(corrMat)
    plt.title(f'Correlation Matrix for {filename}', fontsize=15)
    plt.show()

In [ ]:
# Scatter and density plots
def plotScatterMatrix(df, plotSize, textSize):
    df = df.select_dtypes(include =[np.number]) # keep only numerical columns
    # Remove rows and columns that would lead to df being singular
    df = df.dropna('columns')
    df = df[[col for col in df if df[col].nunique() > 1]] # keep columns where there are more than 1 unique values
    columnNames = list(df)
    if len(columnNames) > 10: # reduce the number of columns for matrix inversion of kernel density plots
        columnNames = columnNames[:10]
    df = df[columnNames]
    ax = pd.plotting.scatter_matrix(df, alpha=0.75, figsize=[plotSize, plotSize], diagonal='kde')
    corrs = df.corr().values
    for i, j in zip(*plt.np.triu_indices_from(ax, k = 1)):
        ax[i, j].annotate('Corr. coef = %.3f' % corrs[i, j], (0.8, 0.2), xycoords='axes fraction', ha='center', va='center', size=textSize)
    plt.suptitle('Scatter and Density Plot')
    plt.show()

In [ ]:
nRowsRead = 1000 # specify 'None' if want to read whole file
# submission.csv may have more rows in reality, but we are only loading/previewing the first 1000 rows
df1 = pd.read_csv('./submission_densenet.csv', delimiter=',', nrows = nRowsRead)
df1.dataframeName = 'submission_densenet.csv'
nRow, nCol = df1.shape
print(f'There are {nRow} rows and {nCol} columns')

In [ ]:
df1.head(5)

In [ ]:
plotPerColumnDistribution(df1, 10, 5)

In [ ]:
plotCorrelationMatrix(df1, 8)

In [ ]:
plotScatterMatrix(df1, 12, 10)

<h2>2. InceptionV3</h2>

In [ ]:
with strategy.scope():
    model = tf.keras.Sequential([InceptionV3(input_shape=(512, 512, 3),
                                             weights='imagenet',
                                             include_top=False),
                                 L.GlobalAveragePooling2D(),
                                 L.Dense(train_labels.shape[1],
                                         activation='softmax')])
        
    model.compile(optimizer='adam',
                  loss = 'categorical_crossentropy',
                  metrics=['categorical_accuracy'])
    model.summary()

In [ ]:
SVG(tf.keras.utils.model_to_dot(Model(model.layers[0].input, model.layers[0].layers[11].output), dpi=70).create(prog='dot', format='svg'))

In [ ]:
SVG(tf.keras.utils.model_to_dot(model, dpi=70).create(prog='dot', format='svg'))

In [ ]:
history = model.fit(train_dataset,
                    epochs=EPOCHS,
                    callbacks=[lr_schedule],
                    steps_per_epoch=STEPS_PER_EPOCH,
                    validation_data=valid_dataset)

In [ ]:
print("Loss of the model is - " , model.evaluate(valid_dataset)[1]*100, "%")
print("Accuracy of the model is - " , model.evaluate(train_dataset)[1]*100 , "%")

In [ ]:
model.save('InceptionV3.hdf5')

In [ ]:
display_training_curves(
    history.history['categorical_accuracy'], 
    history.history['val_categorical_accuracy'], 
    'accuracy')

In [ ]:
#Visualizing train and valid loss
display_training_curves(
    history.history['loss'], 
    history.history['val_loss'], 
    'loss')

In [ ]:
#plotting training values
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

acc = history.history['categorical_accuracy']
val_acc = history.history['val_categorical_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)

#accuracy plot
plt.plot(epochs, acc, color='blue', label='Training Accuracy')
plt.plot(epochs, val_acc, color='red', label='Validation Accuracy')
plt.title('Training and Validation Accuracy with InceptionV3')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend()

plt.figure()
#loss plot
plt.plot(epochs, loss, color='blue', label='Training Loss')
plt.plot(epochs, val_loss, color='red', label='Validation Loss')
plt.title('Training and Validation Loss with InceptionV3')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

In [ ]:
model.evaluate(valid_dataset, verbose=1)

In [ ]:
print(f"Test Accuracy: {probs_incepv3[1]*100}")

In [ ]:
import pickle

In [ ]:
# save the model to disk
print("[INFO] Saving model...")
pickle.dump(model,open('InceptionV3_model.pkl', 'wb'))

In [ ]:
probs_incepv3 = model.predict(test_dataset, verbose=1)
sub.loc[:, 'healthy':] = probs_incepv3
sub.to_csv('submission_incepv3.csv', index=False)
sub.head()



In [ ]:
model.evaluate(test_dataset, verbose=1)
print(f"Test Accuracy: {model.evaluate[1]*100}")

In [ ]:

test_predictions = model.predict(test_dataset)
print(f"Test Accuracy: {test_predictions[1]*100}")

In [ ]:
fig, ax = plt.subplots(nrows=5, ncols=2, figsize=(15, 18))

for i in range(5):
    img_id = random.choice(np.arange(0,test_df.shape[0]))
    test_image = getImage(test_df.image_id[img_id])
    
    ax[i,0].imshow(test_image)
    ax[i,0].set_title(f'{test_df.image_id[img_id]}', fontsize=12)
    ax[i,1].barh(y=train_df.columns[1:],width=test_predictions[img_id])
    ax[i,1].set_title('Predictions', fontsize=12)
    
fig.suptitle("Test set Predictions",fontsize=20)
    
plt.show()

In [ ]:
print(f"Test Accuracy: {scores[1]*100}")

<h2>3. ResNet</h2>

In [ ]:
with strategy.scope():
    model = tf.keras.Sequential([ResNet50V2(input_shape=(512, 512, 3),
                                             weights='imagenet',
                                             include_top=False),
                                 L.GlobalAveragePooling2D(),
                                 L.Dense(train_labels.shape[1],
                                         activation='softmax')])
        
    model.compile(optimizer='adam',
                  loss = 'categorical_crossentropy',
                  metrics=['categorical_accuracy'])
    model.summary()

In [ ]:
SVG(tf.keras.utils.model_to_dot(Model(model.layers[0].input, model.layers[0].layers[11].output), dpi=70).create(prog='dot', format='svg'))

In [ ]:
SVG(tf.keras.utils.model_to_dot(model, dpi=70).create(prog='dot', format='svg'))

In [ ]:
history = model.fit(train_dataset,
                    epochs=EPOCHS,
                    callbacks=[lr_schedule],
                    steps_per_epoch=STEPS_PER_EPOCH,
                    validation_data=valid_dataset)

In [ ]:
print("Loss of the model is - " , model.evaluate(valid_dataset)[1]*100, "%")
print("Accuracy of the model is - " , model.evaluate(train_dataset)[1]*100 , "%")

In [ ]:
model.save('resnet50.hdf5')

In [ ]:
display_training_curves(
    history.history['categorical_accuracy'], 
    history.history['val_categorical_accuracy'], 
    'accuracy')

In [ ]:
#Visualizing train and valid loss
display_training_curves(
    history.history['loss'], 
    history.history['val_loss'], 
    'loss')

In [ ]:
#plotting training values
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

acc = history.history['categorical_accuracy']
val_acc = history.history['val_categorical_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)

#accuracy plot
plt.plot(epochs, acc, color='blue', label='Training Accuracy')
plt.plot(epochs, val_acc, color='red', label='Validation Accuracy')
plt.title('Training and Validation Accuracy with ResNet')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend()

plt.figure()
#loss plot
plt.plot(epochs, loss, color='blue', label='Training Loss')
plt.plot(epochs, val_loss, color='red', label='Validation Loss')
plt.title('Training and Validation Loss with ResNet')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

In [ ]:
probs_resnet = model.predict(test_dataset, verbose=1)
sub.loc[:, 'healthy':] = probs_resnet
sub.to_csv('submission_resnet.csv', index=False)
sub.head()

#LB:-0.94379

In [ ]:
nRowsRead = 1000 # specify 'None' if want to read whole file
# submission.csv may have more rows in reality, but we are only loading/previewing the first 1000 rows
df1 = pd.read_csv('./submission_resnet.csv', delimiter=',', nrows = nRowsRead)
df1.dataframeName = 'submission_resnet.csv'
nRow, nCol = df1.shape
print(f'There are {nRow} rows and {nCol} columns')

In [ ]:
df1.head(5)

In [ ]:
plotPerColumnDistribution(df1, 10, 5)

In [ ]:
plotCorrelationMatrix(df1, 8)

In [ ]:
plotScatterMatrix(df1, 12, 10)

<h2>4. InceptionResNet</h2>

In [ ]:
with strategy.scope():
    model = tf.keras.Sequential([InceptionResNetV2(input_shape=(512, 512, 3),
                                             weights='imagenet',
                                             include_top=False),
                                 L.GlobalAveragePooling2D(),
                                 L.Dense(train_labels.shape[1],
                                         activation='softmax')])
        
    model.compile(optimizer='adam',
                  loss = 'categorical_crossentropy',
                  metrics=['categorical_accuracy'])
    model.summary()

In [ ]:
SVG(tf.keras.utils.model_to_dot(Model(model.layers[0].input, model.layers[0].layers[11].output), dpi=70).create(prog='dot', format='svg'))

In [ ]:
SVG(tf.keras.utils.model_to_dot(model, dpi=70).create(prog='dot', format='svg'))

In [ ]:
history = model.fit(train_dataset,
                    epochs=EPOCHS,
                    callbacks=[lr_schedule],
                    steps_per_epoch=STEPS_PER_EPOCH,
                    validation_data=valid_dataset)

In [ ]:
print("Loss of the model is - " , model.evaluate(valid_dataset)[1]*100, "%")
print("Accuracy of the model is - " , model.evaluate(train_dataset)[1]*100 , "%")

In [ ]:
model.save('InceptionResNet.hdf5')

In [ ]:
display_training_curves(
    history.history['categorical_accuracy'], 
    history.history['val_categorical_accuracy'], 
    'accuracy')

In [ ]:
#Visualizing train and valid loss
display_training_curves(
    history.history['loss'], 
    history.history['val_loss'], 
    'loss')

In [ ]:
#plotting training values
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

acc = history.history['categorical_accuracy']
val_acc = history.history['val_categorical_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)

#accuracy plot
plt.plot(epochs, acc, color='blue', label='Training Accuracy')
plt.plot(epochs, val_acc, color='red', label='Validation Accuracy')
plt.title('Training and Validation Accuracy with InceptionResNet')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend()

plt.figure()
#loss plot
plt.plot(epochs, loss, color='blue', label='Training Loss')
plt.plot(epochs, val_loss, color='red', label='Validation Loss')
plt.title('Training and Validation Loss with InceptionResNet')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

In [ ]:
probs_incepres = model.predict(test_dataset, verbose=1)
sub.loc[:, 'healthy':] = probs_incepres
sub.to_csv('submission_incepres.csv', index=False)
sub.head()

#LB:-0.96181

In [ ]:
nRowsRead = 1000 # specify 'None' if want to read whole file
# submission.csv may have more rows in reality, but we are only loading/previewing the first 1000 rows
df1 = pd.read_csv('./submission_incepres.csv', delimiter=',', nrows = nRowsRead)
df1.dataframeName = 'submission_incepres.csv'
nRow, nCol = df1.shape
print(f'There are {nRow} rows and {nCol} columns')

In [ ]:
df1.head(5)

In [ ]:
plotPerColumnDistribution(df1, 10, 5)

In [ ]:
plotCorrelationMatrix(df1, 8)

In [ ]:
plotScatterMatrix(df1, 12, 10)

In [ ]:
#5. VGG19

In [ ]:
with strategy.scope():
    model = tf.keras.Sequential([VGG19(input_shape=(512, 512, 3),
                                             weights='imagenet',
                                             include_top=False),
                                 L.GlobalAveragePooling2D(),
                                 L.Dense(train_labels.shape[1],
                                         activation='softmax')])
        
    model.compile(optimizer='adam',
                  loss = 'categorical_crossentropy',
                  metrics=['categorical_accuracy'])
    model.summary()

In [ ]:
SVG(tf.keras.utils.model_to_dot(Model(model.layers[0].input, model.layers[0].layers[11].output), dpi=70).create(prog='dot', format='svg'))

In [ ]:
SVG(tf.keras.utils.model_to_dot(model, dpi=70).create(prog='dot', format='svg'))

In [ ]:
history = model.fit(train_dataset,
                    epochs=EPOCHS,
                    callbacks=[lr_schedule],
                    steps_per_epoch=STEPS_PER_EPOCH,
                    validation_data=valid_dataset)

In [ ]:
print("Loss of the model is - " , model.evaluate(valid_dataset)[1]*100, "%")
print("Accuracy of the model is - " , model.evaluate(train_dataset)[1]*100 , "%")

In [ ]:
model.save('VGG19.hdf5')

In [ ]:
display_training_curves(
    history.history['categorical_accuracy'], 
    history.history['val_categorical_accuracy'], 
    'accuracy')

In [ ]:
display_training_curves(
    history.history['loss'], 
    history.history['val_loss'], 
    'loss')

In [ ]:
#plotting training values
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

acc = history.history['categorical_accuracy']
val_acc = history.history['val_categorical_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)

#accuracy plot
plt.plot(epochs, acc, color='blue', label='Training Accuracy')
plt.plot(epochs, val_acc, color='red', label='Validation Accuracy')
plt.title('Training and Validation Accuracy with VGG19')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend()

plt.figure()
#loss plot
plt.plot(epochs, loss, color='blue', label='Training Loss')
plt.plot(epochs, val_loss, color='red', label='Validation Loss')
plt.title('Training and Validation Loss with VGG19')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

In [ ]:
probs_incepres = model.predict(test_dataset, verbose=1)
sub.loc[:, 'healthy':] = probs_incepres
sub.to_csv('submission_vgg19.csv', index=False)
sub.head()


In [ ]:
nRowsRead = 1000 # specify 'None' if want to read whole file
# submission.csv may have more rows in reality, but we are only loading/previewing the first 1000 rows
df1 = pd.read_csv('./submission_vgg19.csv', delimiter=',', nrows = nRowsRead)
df1.dataframeName = 'submission_vgg19.csv'
nRow, nCol = df1.shape
print(f'There are {nRow} rows and {nCol} columns')

In [ ]:
df1.head(5)

In [ ]:
plotPerColumnDistribution(df1, 10, 5)

In [ ]:
plotCorrelationMatrix(df1, 8)

In [ ]:
plotScatterMatrix(df1, 12, 10)

In [ ]:
#6. VGG16

In [ ]:
with strategy.scope():
    model = tf.keras.Sequential([VGG16(input_shape=(512, 512, 3),
                                             weights='imagenet',
                                             include_top=False),
                                 L.GlobalAveragePooling2D(),
                                 L.Dense(train_labels.shape[1],
                                         activation='softmax')])
        
    model.compile(optimizer='adam',
                  loss = 'categorical_crossentropy',
                  metrics=['categorical_accuracy'])
    model.summary()

In [ ]:
SVG(tf.keras.utils.model_to_dot(Model(model.layers[0].input, model.layers[0].layers[11].output), dpi=70).create(prog='dot', format='svg'))

In [ ]:
SVG(tf.keras.utils.model_to_dot(model, dpi=70).create(prog='dot', format='svg'))

In [ ]:
history = model.fit(train_dataset,
                    epochs=EPOCHS,
                    callbacks=[lr_schedule],
                    steps_per_epoch=STEPS_PER_EPOCH,
                    validation_data=valid_dataset)

In [ ]:
print("Loss of the model is - " , model.evaluate(valid_dataset)[1]*100, "%")
print("Accuracy of the model is - " , model.evaluate(train_dataset)[1]*100 , "%")

In [ ]:
model.save('VGG16.hdf5')

In [ ]:
display_training_curves(
    history.history['categorical_accuracy'], 
    history.history['val_categorical_accuracy'], 
    'accuracy')

In [ ]:
display_training_curves(
    history.history['loss'], 
    history.history['val_loss'], 
    'loss')

In [ ]:
#plotting training values
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

acc = history.history['categorical_accuracy']
val_acc = history.history['val_categorical_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)

#accuracy plot
plt.plot(epochs, acc, color='blue', label='Training Accuracy')
plt.plot(epochs, val_acc, color='red', label='Validation Accuracy')
plt.title('Training and Validation Accuracy with VGG16')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend()

plt.figure()
#loss plot
plt.plot(epochs, loss, color='blue', label='Training Loss')
plt.plot(epochs, val_loss, color='red', label='Validation Loss')
plt.title('Training and Validation Loss with VGG16')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

In [ ]:
probs_incepres = model.predict(test_dataset, verbose=1)
sub.loc[:, 'healthy':] = probs_incepres
sub.to_csv('submission_vgg16.csv', index=False)
sub.head()


In [ ]:
nRowsRead = 1000 # specify 'None' if want to read whole file
# submission.csv may have more rows in reality, but we are only loading/previewing the first 1000 rows
df1 = pd.read_csv('./submission_vgg16.csv', delimiter=',', nrows = nRowsRead)
df1.dataframeName = 'submission_vgg16.csv'
nRow, nCol = df1.shape
print(f'There are {nRow} rows and {nCol} columns')

In [ ]:
df1.head(5)

In [ ]:
plotPerColumnDistribution(df1, 10, 5)

In [ ]:
plotCorrelationMatrix(df1, 8)

In [ ]:
plotScatterMatrix(df1, 12, 10)